In [54]:
# http://andybromberg.com/sentiment-analysis-python/
# Andy Bromberg's Simple Sentiment Analysis System
# Uses data from Pang & Lee (2005)
# Uses a Naive Bayes Classifier Train the System
#  NB Updated 2016 for package changes around scores

import re, math, collections, itertools, sys, os
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.metrics import BigramAssocMeasures, scores
from nltk.probability import FreqDist, ConditionalFreqDist

In [60]:
def TagWords(sentence):
    a = list()
    try:
            tagged = nltk.pos_tag(sentence)
            for item in tagged:
                if item[1][0] == 'U': #interjection
                      a.append(item[0])
                elif item[1][0] =='V': #verbs
                    a.append(item[0])
                elif item [1][0] == 'R': #adverbs
                    a.append(item[0])
                elif item [1][0] == 'J': #adjectives
                    a.append(item[0])
    except Exception as e:
        print(str(e))
    print(a)
    return (a)

In [61]:
#function extracts all verbs, adjectives,interjections and adverbs from a list
def get_v_adj_adv_interjection(text):
    picked_words = []
    for item in text:
        if item[1] == 'UH':#interjection
            print ('ITEM IS', item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'JJ':#j is adj
            print (item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'JJR':
            print (item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'JJS':
            print (item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'RB':
            print (item)#rb is adv
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'RBR':
            print (item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'RBS':
            print ('item is', item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'VB':
            print (item)#base verb
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'VBD':
            print (item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'VBG':
            print (item)
            picked_words.append(item[0])
        elif item [1] == 'VBN':
            print (item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item[1]== 'VBP':
            print (item)
            print('word is ', item[0])
            picked_words.append(item[0])
        elif item [1] == 'VBZ':
            print(item)
            print('word is ', item[0])
            picked_words.append(item[0])
        print ('words are', picked_words)
    return picked_words

In [64]:
def evaluate_features(feature_select):
    #reading pre-labeled input and splitting into lines
    negSentences = open('rt-polarity-neg.txt', 'r', encoding='utf8')
    posSentences = open('rt-polarity-pos.txt', 'r', encoding='utf8')
    negSentences = re.split(r'\n', negSentences.read())
    posSentences = re.split(r'\n', posSentences.read())
    stops = set(stopwords.words("english"))
    posFeatures = []
    negFeatures = []
    # breaks up the sentences into lists of individual words
    # creates instance structures for classifier
    for i in posSentences:
        posWords = re.findall(r"[\w']+|[.,!?;]", i)
        #posWords = [word for word in posWords if word not in stops] #take out stop words
        posWords = TagWords(posWords)
        posWords = [feature_select(posWords), 'pos']
        posFeatures.append(posWords)
    for i in negSentences:
        negWords = re.findall(r"[\w']+|[.,!?;]", i)
        #negWords= [word for word in negWords if word not in stops]
        negWords = TagWords(negWords)
        negWords = [feature_select(negWords), 'neg']
        negFeatures.append(negWords)
        
    posCutoff = int(math.floor(len(posFeatures)*3/4))
    negCutoff = int(math.floor(len(negFeatures)*3/4))
    trainFeatures = posFeatures[:posCutoff] + negFeatures[:negCutoff]
    testFeatures = posFeatures[posCutoff:] + negFeatures[negCutoff:]
    
    #Runs the classifier on the testFeatures
    classifier = NaiveBayesClassifier.train(trainFeatures)
    
    #Sets up labels to look at output
    referenceSets = collections.defaultdict(set)
    testSets = collections.defaultdict(set)
    for i, (features, label) in enumerate(testFeatures): # enumerate adds number-count to each item
        referenceSets[label].add(i)               # recorded polarity for these test sentences
        predicted = classifier.classify(features) # classifiers' proposed polarity for tests
        testSets[predicted].add(i)

    #Outputs
    print('train on %s instances, test on %s instances'% (len(trainFeatures), len(testFeatures)))
    print('accuracy:', nltk.classify.util.accuracy(classifier, testFeatures))
    print('pos precision:', scores.precision(referenceSets['pos'], testSets['pos']))
    print('pos recall:', scores.recall(referenceSets['pos'], testSets['pos']))
    print('neg precision:', scores.precision(referenceSets['neg'], testSets['neg']))
    print('neg recall:', scores.recall(referenceSets['neg'], testSets['neg']))
    classifier.show_most_informative_features(10)

def make_full_dict(words):
    return dict([(word, True) for word in words])

print('using all words as features')
evaluate_features(make_full_dict)


using all words as features
['is', 'destined', 'be', "century's", 'new', 'going', 'make', 'even', 'greater', 'arnold', 'jean', 'steven']
['gorgeously', 'elaborate', 'is', 'so', 'huge', 'cannot', 'adequately', 'describe', 'co', 'expanded', 'middle']
['effective', 'too', 'tepid']
['sometimes', 'go', 'have', 'is', 'good', 'start']
['rare', 'so', 'honest', 'keenly', 'observed', "doesn't", 'feel']
['provides', 'great', 'neurotic', 'even', 'have', 'reached', 'absolute']
['rare']
['perhaps', 'ever', 'made', 'has', 'more', 'literally', 'showed', 'hell', 'is', 'paved', 'good']
['turns', 'snappy', 'curls', 'so', 'clever', 'want', 'hate', 'somehow', 'pulls', 'off']
['take', 'offers', 'refreshingly', 'different', 'asian']
['is', 'well', 'seeing', 'talking', 'singing']
['really', 'surprises', 'is', 'low', 'key', 'genuine']
['is', 'go', 'be', 'fed']
['greatest', 'oriented', 'fantasy', 'ever']
['ultimately', 'ponders', 'need', 'so', 'much']
['utterly', 'wrote', 'most', 'famous', 'ever', 'lived', 'com

['pay', 'hear', 'suffering', 'afghan', 'still', 'be', 'unaffected', 'make', 'human']
['thunderous', 'first', 'quiet', 'are', 'few', 'far', 'dilutes', 'otherwise', 'respectable', 'still', 'is', 'fun', 'truly', 'excellent']
['obviously', 'struck', 'responsive', 'many', 'work', 'other']
['see', 'bracing', 'big']
['classy', 'have', 'left', 'prove', 'still', 'has', 'show', 'done']
['is', 'nature', "fessenden's", 'has', 'proved', 'important', 'is', 'especially', 'so']
['not', 'exactly', 'gourmet', 'is', 'fair', 'even', 'coming']
['is', 'was', 'made', 'shut', 'out', 'real', 'take', 'vicarious', 'last', 'frontier']
['merely', 'technical', 'logistical', 'russian', 'marks', 'cinematic']
['is', 'talented', 'terribly', 'charismatic', 'essential', 'social']
["it's", 'great', 'very', 'little', 'spectacular', 'is', 'is', 'infectious']
['original']
['delight', 'know']
['gloriously', 'gory']
['overcomes', 'regular', 'coming', 'potent']
["haven't", 'been', 'dulled', 'slasher', 'psychological', 'is']
["i

['strong', "it's", 'dong', 'remember']
['is', 'brimming', 'speaks', 'human', 'find', 'easily', 'crush', 'forever']
['steven', 'has', 'succeeded', 'focusing', 'intently', 'making', 'quirky', 'rather']
['ultimately', 'stands', 'important', 'latin', 'most', 'oppressive']
['has', 'soft', 'percolating']
['made', 'often', 'lovely']
['using', 'figurative', 'dong', 'pulls', 'even', 'handed', 'ideological', 'unloading', 'continues', 'still', 'ahead']
['understands', 'defines', 'as', 'much', 'defines']
['is', 'lively', 'as', 'is', 'unapologetically', 'dumb']
['several', 'greek', 'american', 'happily', 'testify', 'comparative']
['has', 'ever', 'been', 'possible', 'say', 'has', 'truly', 'inhabited', 'is', 'now']
['presenting', 'impossible', 'impossible', 'pumpkin', 'say', 'is', 'impossible', 'forces', 'confront', "what's", 'possible', 'do', 'make', 'so']
['impressive', 'first', 'especially', 'considering', 'is']
['incendiary', 'deeply', 'thought', 'provoking', 'most', 'peculiar', 'peculiarly', 've

['go', 'see', 'enjoy']
['impress', 'even', 'have', 'little']
['proves', "he's", 'quite', 'talented', 'shows', "he's", 'dangerous']
["there's", 'implied', 'is', 'just']
['murderous', 'well', 'be', 'most', 'comprehensive', 'also', 'closest']
['are', 'so', 'funny', 'aggressive', 'alive', 'have', 'watch', "can't", 'wait', 'see', 'do', 'next']
['green', 'go', "it's"]
['reilly', 'burke', 'octopus', 'has', 'just', 'right']
['deceivingly', 'simple', 'grows']
['is', 'vibrant', 'makes', 'successful', 'accessible']
['skillful', 'little']
['very', 'witty', 'uses', 'humour', 'make']
['uses', 'miss', 'aesthetic', 'often', 'enough', 'keep', 'entertaining', 'even', 'makes']
['is', 'wonderful', 'bravura', "doesn't", 'deserve', 'leave', 'is', 'aware']
['long', 'never', 'feels', 'screams']
['does', 'good', 'here', 'working', 'natural']
['precious', 'artfully', 'camouflaged', 'everyday']
['kinnear', 'gives', 'tremendous']
['best', 'lucas', 'take', 'is', 'use', 'special']
['has', 'been', 'written', 'so', '

['too', 'sincere', 'exploit', 'too', 'honest', 'manipulate']
["glass'", 'evocative', 'ultimately', 'leaves', 'divining']
['serious', 'finds', 'lay', 'bare', 'setting', 'good']
['depressing', "don't", 'object', 'unelected', 'have', 'suspected', 'is', 'incurious', 'uncharismatic', 'overgrown', 'mean', 'mile', 'wide']
['gives', 'emotional', "you'll", 'be', 'glad', 'went', 'along']
['sure', "it's", 'more', 'same', 'proves', "that's", 'not', 'always', 'bad']
['lighthearted', 'feel', 'good', 'embraces', 'honored', 'most', 'powerful', 'is', 'love']
['stopping']
['is', 'be', 'seen', 'especially', "aren't", 'have', 'forgotten', 'unmentioned']
['warm', 'genuine', 'lie', 'not', 'genuinely', 'believe', "it's", 'only', 'bring', 'loved']
['surprising', 'new', 'familiar', 'amalgamating', 'adding', 'true', 'human', 'not']
['hokey', 'super', 'definitely', 'not', 'sit', 'back', 'have', 'few', 'little', 'get', 'fuzzy']
['pleasant', 'romantic']
["it's"]
['has', 'delivered', 'undoubted', 'stylistic', 'has'

...

['most', 'excruciating', 'sit', 'do', 'not', 'involve']
['was', 'more', 'tepid', 'trotting', 'out', 'worked', 'ago', 'has', 'lost']
['goes', 'too', 'long', 'down', 'unnecessary']
['absolutely', 'amazing', 'first', 'kevin', 'donovan', 'managed', 'find', 'new', 'add', 'make', "chan's"]
['get', 'good', 'derailed', 'seek', 'strike', 'just', 'right']
["'in", 'poor', 'such', 'well', 'loved', 'classic', 'exposes', 'basic']
["it's", 'old', 'cheap']
["film's", 'final', 'nearly', 'previous', 'unseen', 'is', 'unconvincing', 'was', 'right', 'cut']
['does', 'excellent', 'critiquing', 'faltering', 'half', 'criticizing', 'more', 'commiserating']
['found', 'slow', 'predictable', 'not', 'very', 'amusing']
['seems', 'far', 'more', 'interested', 'gross', 'showing', 'well', 'thought', "haven't", 'seen']
['need', 'muster', 'just', 'figuring', 'out', 'pretentious']
['sized', "'goodfellas'", 'designed', 'appeal', 'younger', "it's", 'not', 'very', 'good', 'objective', 'does', 'mostly', 'hold', "one's"]
['get'

['lurid', 'less', 'lucid']
['wannabe', 'brainy', 'prep', 'complex', 'own', 'squanders', 'beautiful']
['brief', 'need', 'more', 'x', 'less']
['see', 'black', 'camps', 'up', 'named', 'dirty']
['pass', 'true']
['is', 'surely', 'most', 'frantic', 'virulent', 'foul', 'natured', 'ever', 'delivered']
['once', 'has', 'been', 'quashed', 'is', 'even', 'funniest']
['porn']
['connected', 'are', 'actually', 'fascinating', 'invincible', 'is', 'such', 'does', 'not', 'do']
['depressingly', "feminist'", 'romantic', 'takes', 'astonishingly', 'condescending']
['never', 'land', 'is', 'much', 'more', 'original', 'more', 'racist', 'is', 'missing']
['just', "don't", 'cold', 'hearted', 'be', 'reno', 'gets', 'just', 'bring']
["it's", 'quite', 'tasteful', 'look', 'tends', 'do', 'little', 'own']
['is', 'shallow', 'offensive', 'redundant', 'pitifully', 'few', 'real']
['swim', 'is', 'merely', 'anne', "birot's", 'off', 'handed', 'saying', 'find', 'difficult', 'wade']
['uses', 'quick', 'very', 'large', 'wide', 'take

['is', 'substitute', "doesn't", 'really', 'know', 'care', 'uses', 'preordained']
[]
['delete']
['is', 'not', 'bad', 'just', "doesn't", 'have', 'really', 'interesting', 'say']
['once', 'starts', 'learning', 'compromise', 'enough', 'become', 'comparatively', 'sane', 'healthy', 'becomes', 'predictably', 'conventional']
['hopefully', "it'll", 'be', 'see', 'then']
['disguising', 'worst']
['lacks', 'previous', 'makes', 'even', 'more', 'unmentionable', 'seem', 'mere', 'splashing', 'around']
['hastily', 'mounted', 'exists', 'only', 'capitalize', "hopkins'", 'play', 'hannibal', 'again', 'even', 'has', 'immediate', 'provide', 'fourth']
['smoochy', 'not', 'nearly', 'as', 'nasty', 'thinks', 'is', 'joke', 'over', 'again']
['implied', 'is', 'terrifying', 'keep', 'coming']
['never', 'gets', 'over', 'own', 'conventional']
['call', 'only', 'be', 'flatter']
['more', 'disposable', "barbera's", 'half', 'ever', 'were']
['falls', 'short', 'spiritual', 'have', 'made', 'original', 'new', 'so', 'compelling']
[

['better', 'described', 'gone', 'badly', 'awry']
['bad', 'superficially', 'written', 'ramble', 'tediously', 'struggling', 'create']
['are', 'playing', 'big', 'new', 'mock', "don't", 'understand', 'figure', "don't", 'care', 'understand']
['competently', 'directed', 'terminally', 'cute']
['big', 'is', 'getting', 'excited', 'then', 'biting', 'finding', 'missing']
['not', 'just', 'unlikable', 'disturbing', 'disgusting', 'redeeming']
['is', 'virtually', 'unwatchable']
['eternally', 'devoted', 'black', 'have', 'intermittently', 'good', 'feel', 'free', 'go', 'get', 'popcorn', 'not', 'onscreen']
['serious', 'makes', 'too', 'well', "isn't", 'much']
['exhausted', 'desiccated', "can't", 'get', 'own']
['main', 'are', 'simply', 'named', 'emphasizing', 'disappointingly', 'generic', 'entire']
['is', 'careless', 'unfocused']
['sticking', 'up', 'fresh', 'now', 'then']
['only', 'glossy', 'are', 'formulaic']
['are', 'thin', 'undergo', 'radical']
['na', 'punctuated', 'graphic']
['is', 'run', 'visual']
['s

["it's", 'be', 'released']
['absolutely', 'unintentionally', 'terrifying']
['eckstraordinarily', 'severely', 'boring']
['legged', 'falls', 'flat']
['much', 'runs', 'around', 'acts', 'accepting', 'old', 'is', 'creepy']
['just', 'have', 'wondering', "lee's", 'just', 'go', 'save']
['crossing', 'over', 'mumbo', 'jumbo', 'manipulative', 'sappy']
['lighten', 'heavy', 'subject', 'silberling', 'also', 'certain', 'trivializes', 'too', 'many', 'nervous']
['has', 'brilliant', 'charismatic', 'suffers', 'rampant']
['uncomfortably', 'close', 'pro', 'serb']
['staggeringly', 'dreadful']
['like', 'high', 'flog', 'dead', 'were', 'surprising', 'trying', 'new']
['final', 'seen']
['throwing', 'pulling', 'seems', 'have', 'ransacked', 'old', 'ii', 'overly', 'familiar']
['few', 'good', 'told', "can't", 'handle', 'high', 'poetically', "don't", 'care']
['further', 'sad', 'spinning', 'run', 'lola', 'has', 'turned', 'out', 'be', 'softheaded', 'metaphysical']
["you'll", 'trudge', 'rode', 'eating', 'extra', 'large'

In [32]:
#empty array#
contentArray =['Starbucks is doing very well lately.',
               'Overall, while it may seem there is already a Starbucks on every corner, Starbucks still has a lot of room to grow.',
               'They just began expansion into food products, which has been going quite well so far for them.',
               'I can attest that my own expenditure when going to Starbucks has increased, in lieu of these food products.',
               'Starbucks is also indeed expanding their number of stores as well.',
               'Starbucks still sees strong sales growth here in the united states, and intends to actually continue increasing this.',
               'Starbucks also has one of the more successful loyalty programs, which accounts for 30%  of all transactions being loyalty-program-based.',
               'As if news could not get any more positive for the company, Brazilian weather has become ideal for producing coffee beans.',
               'Brazil is the world\'s #1 coffee producer, the source of about 1/3rd of the entire world\'s supply!',
               'Given the dry weather, coffee farmers have amped up production, to take as much of an advantage as possible with the dry weather.',
               'Increase in supply... well you know the rules...',]


exampleArray = ['The incredibly intimidating NLP scares people away who are sissies']
               

def processContent(sentence):
    try:
        for item in sentence:
            tokenized = nltk.word_tokenize(item)
            tagged = nltk.pos_tag(tokenized)

            print(tagged)

    except Exception as e:
        print(str(e))
    return (tagged)


x = processContent(exampleArray)

[('The', 'DT'), ('incredibly', 'RB'), ('intimidating', 'VBG'), ('NLP', 'NNP'), ('scares', 'NNS'), ('people', 'NNS'), ('away', 'RB'), ('who', 'WP'), ('are', 'VBP'), ('sissies', 'NNS')]
